In [1]:
import sys
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# 현재 노트북 파일의 경로를 기준으로 프로젝트 루트 경로를 계산
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..'))

# sys.path에 프로젝트 루트 경로가 없으면 추가
if project_root not in sys.path:
  sys.path.append(project_root)


In [2]:
from src.data.db_handler import DBHandler

db_handler = DBHandler(db_name="data_lake")

df = db_handler.fetch_data(table_name="kr_stock_estimate_perform")

column_index = df.columns

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   ticker                   155 non-null    object             
 1   period                   155 non-null    object             
 2   analyst                  155 non-null    object             
 3   opinion                  155 non-null    object             
 4   revenue                  155 non-null    float64            
 5   revenue_yoy              155 non-null    float64            
 6   operating_profit         155 non-null    float64            
 7   operating_profit_yoy     155 non-null    float64            
 8   net_income               155 non-null    float64            
 9   net_income_yoy           155 non-null    float64            
 10  eps                      155 non-null    float64            
 11  eps_yoy                  0 non-n

,ticker,period,analyst,opinion,revenue,revenue_yoy,operating_profit,operating_profit_yoy,net_income,net_income_yoy,eps,eps_yoy,bps,per,pbr,psr,roe,ebitda,ev_ebitda,debt_ratio,interest_coverage_ratio,updated_at
0,000120,2022.12,최고운,매수,121307.0,69.0,4118.0,197.0,1816.0,2316.0,9239.0,None,3449.0,81900.0,114.0,35.0,54.0,None,51.0,None,None,2025-09-21 16:15:28.072191+00:00
1,000120,2023.12,최고운,매수,117679.0,-30.0,4802.0,166.0,2248.0,238.0,10594.0,None,265.0,103570.0,123.0,31.0,55.0,None,63.0,None,None,2025-09-21 16:15:28.072191+00:00
2,000120,2024.12,최고운,매수,121168.0,30.0,5307.0,105.0,2485.0,105.0,11525.0,None,167.0,120880.0,70.0,33.0,43.0,None,66.0,None,None,2025-09-21 16:15:28.072191+00:00
3,000120,2025.12E,최고운,매수,124225.0,25.0,4966.0,-64.0,2483.0,-1.0,11228.0,None,30.0,124513.0,75.0,30.0,46.0,None,61.0,None,None,2025-09-21 16:15:28.072191+00:00
4,000120,2026.12E,최고운,매수,130454.0,50.0,5657.0,139.0,2949.0,188.0,12090.0,None,188.0,147865.0,63.0,35.0,41.0,None,68.0,None,None,2025-09-21 16:15:28.072191+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,259960,2022.12,정호윤,매수,18540.0,-17.0,7516.0,155.0,5002.0,-38.0,8590.0,None,-113.0,106660.0,158.0,992.0,63.0,None,103.0,None,None,2025-09-21 16:15:28.072191+00:00
151,259960,2023.12,정호윤,매수,19106.0,31.0,7680.0,22.0,5954.0,190.0,8761.0,None,201.0,128120.0,151.0,862.0,74.0,None,112.0,None,None,2025-09-21 16:15:28.072191+00:00
152,259960,2024.12,정호윤,매수,27098.0,418.0,11825.0,540.0,13061.0,1194.0,12889.0,None,1232.0,285930.0,109.0,1250.0,87.0,None,211.0,None,None,2025-09-21 16:15:28.072191+00:00
153,259960,2025.12E,정호윤,매수,32074.0,184.0,14097.0,192.0,9374.0,-282.0,15158.0,None,-282.0,205219.0,160.0,4333.0,73.0,None,128.0,None,None,2025-09-21 16:15:28.072191+00:00


In [3]:
final_cols = column_index

In [4]:
df[final_cols] = df[final_cols].replace(r'^\s*$', np.nan, regex=True)
df[final_cols] = df[final_cols].replace(['0', 0], np.nan)

# 결측치 개수 계산
missing_value_counts = df[final_cols].isnull().sum()

# 전체 행의 개수 계산
total_rows = len(df[final_cols])

# 결측치 비율 계산
missing_value_percent = ((missing_value_counts / total_rows) * 100).round(2)

# --- 3. 결과 취합 및 출력 ---
missing_df = pd.DataFrame({
  'missing_count': missing_value_counts,
  'missing_percent': missing_value_percent
})

# 결측치가 많은 순서대로 정렬하여 출력
missing_df = missing_df[missing_df['missing_count'] > 0]
missing_df = missing_df.sort_values(by='missing_count', ascending=False)

missing_df

,missing_count,missing_percent
eps_yoy,155,100.00
ebitda,155,100.00
debt_ratio,155,100.00
interest_coverage_ratio,155,100.00
pbr,15,9.68
eps,1,0.65


In [5]:
missing_cols = missing_df[missing_df['missing_percent'] >= 50].index.to_list()

final_cols = [col for col in final_cols if col not in missing_cols]

print(f"결측 컬럼 : {missing_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] = [후보 컬럼 : {len(final_cols)}]")

결측 컬럼 : ['eps_yoy', 'ebitda', 'debt_ratio', 'interest_coverage_ratio']
[원본 컬럼 : 21] - [결측 컬럼 : 4] = [후보 컬럼 : 18]


In [6]:
nunique_counts = df[final_cols].nunique()

# 전체 행의 개수 계산
total_rows = len(df[final_cols])

# 결측치 비율 계산
nunique_value_percent = ((nunique_counts / total_rows) * 100).round(2)

unique_cols = nunique_counts[nunique_counts == 1].index.tolist()

# --- 3. 결과 취합 및 출력 ---
nunique_df = pd.DataFrame({
  'nunique_count': nunique_counts,
  'nunique_percent': nunique_value_percent
})

# 결측치가 많은 순서대로 정렬하여 출력
nunique_df = nunique_df[nunique_df['nunique_count'] > 1]
nunique_df = nunique_df.sort_values(by='nunique_count', ascending=False)

nunique_df

,nunique_count,nunique_percent
net_income,155,100.00
revenue,155,100.00
operating_profit,155,100.00
per,155,100.00
net_income_yoy,153,98.71
eps,152,98.06
operating_profit_yoy,151,97.42
bps,150,96.77
revenue_yoy,131,84.52
psr,121,78.06


In [7]:
final_cols = [col for col in final_cols if col not in unique_cols]

print(f"유니크 컬럼 : {unique_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] - [유니크 컬럼 : {len(unique_cols)}] = [후보 컬럼 : {len(final_cols)}]")

유니크 컬럼 : ['updated_at']
[원본 컬럼 : 21] - [결측 컬럼 : 4] - [유니크 컬럼 : 1] = [후보 컬럼 : 17]


In [8]:
df[final_cols]

,ticker,period,analyst,opinion,revenue,revenue_yoy,operating_profit,operating_profit_yoy,net_income,net_income_yoy,eps,bps,per,pbr,psr,roe,ev_ebitda
0,000120,2022.12,최고운,매수,121307.0,69.0,4118.0,197.0,1816.0,2316.0,9239.0,3449.0,81900.0,114.0,35.0,54.0,51.0
1,000120,2023.12,최고운,매수,117679.0,-30.0,4802.0,166.0,2248.0,238.0,10594.0,265.0,103570.0,123.0,31.0,55.0,63.0
2,000120,2024.12,최고운,매수,121168.0,30.0,5307.0,105.0,2485.0,105.0,11525.0,167.0,120880.0,70.0,33.0,43.0,66.0
3,000120,2025.12E,최고운,매수,124225.0,25.0,4966.0,-64.0,2483.0,-1.0,11228.0,30.0,124513.0,75.0,30.0,46.0,61.0
4,000120,2026.12E,최고운,매수,130454.0,50.0,5657.0,139.0,2949.0,188.0,12090.0,188.0,147865.0,63.0,35.0,41.0,68.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,259960,2022.12,정호윤,매수,18540.0,-17.0,7516.0,155.0,5002.0,-38.0,8590.0,-113.0,106660.0,158.0,992.0,63.0,103.0
151,259960,2023.12,정호윤,매수,19106.0,31.0,7680.0,22.0,5954.0,190.0,8761.0,201.0,128120.0,151.0,862.0,74.0,112.0
152,259960,2024.12,정호윤,매수,27098.0,418.0,11825.0,540.0,13061.0,1194.0,12889.0,1232.0,285930.0,109.0,1250.0,87.0,211.0
153,259960,2025.12E,정호윤,매수,32074.0,184.0,14097.0,192.0,9374.0,-282.0,15158.0,-282.0,205219.0,160.0,4333.0,73.0,128.0


In [9]:
print(final_cols)

['ticker', 'period', 'analyst', 'opinion', 'revenue', 'revenue_yoy', 'operating_profit', 'operating_profit_yoy', 'net_income', 'net_income_yoy', 'eps', 'bps', 'per', 'pbr', 'psr', 'roe', 'ev_ebitda']


| 컬럼명 | 한글컬럼명 | 중요도 | 이유 |
| :--- | :--- | :--- | :--- |
| `ticker` | 종목 코드 | **필수** | 어떤 기업의 컨센서스인지를 특정하는 핵심 식별자입니다. |
| `period` | 분석기간 | **필수** | 데이터가 어느 시점의 실적(또는 추정치)인지를 나타내는 기준점으로, 시계열 분석에 필수적입니다. 'E'는 추정치를 의미합니다. |
| `analyst` | 애널리스트명 | **하** | 특정 애널리스트의 성향을 분석하는 모델이 아니라면, 일반적인 변동성 모델에서는 개별 애널리스트 이름의 중요도는 낮습니다. |
| `opinion` | 투자의견 | **상** | '매수', '중립' 등의 투자의견은 시장 심리를 직접적으로 반영하는 중요한 범주형 피처입니다. 의견의 상향/하향 조정은 주가에 큰 영향을 줍니다. |
| `revenue` | 매출액 | **상** | 기업의 외형적 성장에 대한 시장의 기대를 나타내는 핵심 지표입니다. 실제 발표된 실적이 이 추정치와 얼마나 차이 나는지가 중요합니다. |
| `revenue_yoy` | 매출액(전년비) | **상** | 단순 매출액보다 성장률을 통해 기업의 성장 모멘텀을 파악할 수 있어 더 중요한 정보가 될 수 있습니다. |
| `operating_profit`| 영업이익 | **필수** | 기업의 수익성에 대한 시장의 기대를 보여주는 가장 중요한 지표 중 하나입니다. 어닝 서프라이즈/쇼크의 핵심 판단 기준이 됩니다. |
| `operating_profit_yoy`| 영업이익(전년비) | **필수** | 영업이익의 성장률은 기업의 수익성 개선 모멘텀을 나타내므로, 단순 이익 규모보다 주가에 더 민감하게 반응하는 경우가 많습니다. |
| `net_income` | 순이익 | **상** | 기업의 최종적인 이익에 대한 시장 추정치로, PER 계산의 기반이 되는 중요한 데이터입니다. |
| `net_income_yoy`| 순이익(전년비) | **상** | 순이익의 성장률 또한 수익성 모멘텀을 판단하는 데 있어 중요한 참고 지표가 됩니다. |
| `eps` | EPS(주당순이익) | **필수** | 주식 1주당 순이익 추정치로, PER 계산 및 기업 가치 평가의 가장 핵심적인 요소입니다. |
| `bps` | BPS(주당순자산)| **상** | 주당순자산 추정치로, PBR 계산의 기반이 되며 기업의 장부 가치 및 안정성에 대한 시장의 평가를 보여줍니다. |
| `per` | PER(주가수익비율)| **상** | 이익 대비 주가 수준을 나타내는 대표적인 가치평가(밸류에이션) 지표입니다. 시장의 고평가/저평가 인식을 보여줍니다. |
| `pbr` | PBR(주가순자산비율)| **상** | 순자산 대비 주가 수준을 나타내는 가치평가 지표로, 특히 기업의 자산가치 대비 주가 수준을 판단할 때 중요합니다. |
| `psr` | PSR(주가매출비율)| **중** | 매출 대비 주가 수준을 나타냅니다. 아직 이익을 내지 못하는 성장주나 기술주의 가치 평가에 주로 사용됩니다. |
| `roe` | ROE(자기자본이익률)| **상** | 자기자본 대비 이익률 추정치로, 기업의 수익성 및 효율성에 대한 시장의 기대를 반영하는 핵심 지표입니다. |
| `ev_ebitda` | EV/EBITDA | **중** | 기업의 현금흐름 창출 능력 대비 기업가치를 나타내는 지표로, 특히 M&A나 산업 분석에서 중요하게 사용됩니다. |

In [10]:
final_cols = [
  'ticker',
  'period',
  'opinion',
  'revenue',
  'revenue_yoy',
  'operating_profit',
  'operating_profit_yoy',
  'net_income',
  'net_income_yoy',
  'eps',
  'bps',
  'per',
  'pbr',
  'psr',
  'roe',
  'ev_ebitda'
]

In [11]:
df[final_cols]

,ticker,period,opinion,revenue,revenue_yoy,operating_profit,operating_profit_yoy,net_income,net_income_yoy,eps,bps,per,pbr,psr,roe,ev_ebitda
0,000120,2022.12,매수,121307.0,69.0,4118.0,197.0,1816.0,2316.0,9239.0,3449.0,81900.0,114.0,35.0,54.0,51.0
1,000120,2023.12,매수,117679.0,-30.0,4802.0,166.0,2248.0,238.0,10594.0,265.0,103570.0,123.0,31.0,55.0,63.0
2,000120,2024.12,매수,121168.0,30.0,5307.0,105.0,2485.0,105.0,11525.0,167.0,120880.0,70.0,33.0,43.0,66.0
3,000120,2025.12E,매수,124225.0,25.0,4966.0,-64.0,2483.0,-1.0,11228.0,30.0,124513.0,75.0,30.0,46.0,61.0
4,000120,2026.12E,매수,130454.0,50.0,5657.0,139.0,2949.0,188.0,12090.0,188.0,147865.0,63.0,35.0,41.0,68.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,259960,2022.12,매수,18540.0,-17.0,7516.0,155.0,5002.0,-38.0,8590.0,-113.0,106660.0,158.0,992.0,63.0,103.0
151,259960,2023.12,매수,19106.0,31.0,7680.0,22.0,5954.0,190.0,8761.0,201.0,128120.0,151.0,862.0,74.0,112.0
152,259960,2024.12,매수,27098.0,418.0,11825.0,540.0,13061.0,1194.0,12889.0,1232.0,285930.0,109.0,1250.0,87.0,211.0
153,259960,2025.12E,매수,32074.0,184.0,14097.0,192.0,9374.0,-282.0,15158.0,-282.0,205219.0,160.0,4333.0,73.0,128.0


In [12]:
final_df = df[final_cols].copy()

# --- 1단계: 날짜 타입 자동 변환 ---
remaining_object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in remaining_object_cols:
  original_valid_count = final_df[col].count()
  if original_valid_count == 0: continue
  
  for format in ['%Y%m', '%Y%m%d']:
    date_series = pd.to_datetime(final_df[col], format=format,errors='coerce')
    success_rate = date_series.count() / original_valid_count
    
    if success_rate > 0.95: # 95% 임계값
      final_df[col] = date_series
      break

# --- 2단계: 저비율 카테고리 자동 변환 ---
object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in object_cols:
  ratio = final_df[col].nunique() / len(final_df[col])
  if ratio < 0.05: # 5% 임계값
    final_df[col] = final_df[col].astype('category')


In [13]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   ticker                155 non-null    object  
 1   period                155 non-null    category
 2   opinion               155 non-null    category
 3   revenue               155 non-null    float64 
 4   revenue_yoy           155 non-null    float64 
 5   operating_profit      155 non-null    float64 
 6   operating_profit_yoy  155 non-null    float64 
 7   net_income            155 non-null    float64 
 8   net_income_yoy        155 non-null    float64 
 9   eps                   154 non-null    float64 
 10  bps                   155 non-null    float64 
 11  per                   155 non-null    float64 
 12  pbr                   140 non-null    float64 
 13  psr                   155 non-null    float64 
 14  roe                   155 non-null    float64 
 15  ev_ebi

| 컬럼명 | 한글컬럼명 | 현재 타입 | 적정 타입 | 이유 및 해결 방안 |
| :--- | :--- | :--- | :--- | :--- |
| `ticker` | 종목 코드 | `object` | `category` | 반복되는 종목 코드는 메모리 효율을 위해 `category` 타입으로 변환하는 것이 가장 효율적입니다. |
| `period` | 분석기간 | `category` | `category` | '2024.12E'와 같이 문자열 패턴을 가지므로 `category` 타입이 적합합니다. |
| `opinion` | 투자의견 | `category` | `category` | '매수', '중립' 등 한정된 값이므로 `category` 타입이 적합합니다. |
| `revenue` | 매출액 | `float64` | `float64` | 금액 데이터이며 소수점을 포함하므로 `float64`가 적합합니다. |
| `revenue_yoy` | 매출액(전년비) | `float64` | `float64` | 비율 데이터이므로 `float64`가 적합합니다. |
| `operating_profit`| 영업이익 | `float64` | `float64` | 금액 데이터이며 음수 및 소수점을 포함하므로 `float64`가 적합합니다. |
| `operating_profit_yoy`| 영업이익(전년비) | `float64` | `float64` | 비율 데이터이므로 `float64`가 적합합니다. |
| `net_income` | 순이익 | `float64` | `float64` | 금액 데이터이며 음수 및 소수점을 포함하므로 `float64`가 적합합니다. |
| `net_income_yoy`| 순이익(전년비) | `float64` | `float64` | 비율 데이터이므로 `float64`가 적합합니다. |
| `eps` | EPS(주당순이익) | `float64` | `float64` | 소수점을 포함할 수 있으므로 `float64`가 안전합니다. |
| `bps` | BPS(주당순자산)| `float64` | `float64` | 소수점을 포함할 수 있으므로 `float64`로 관리하는 것이 좋습니다. |
| `per` | PER(주가수익비율)| `float64` | `float64` | 밸류에이션 지표는 소수점 및 음수 표현이 필요하므로 `float64`가 적합합니다. |
| `pbr` | PBR(주가순자산비율)| `float64` | `float64` | 밸류에이션 지표이므로 `float64` 타입이 적합합니다. |
| `psr` | PSR(주가매출비율)| `float64` | `float64` | 밸류에이션 지표이므로 `float64` 타입이 적합합니다. |
| `roe` | ROE(자기자본이익률)| `float64` | `float64` | 수익률 지표이므로 `float64` 타입이 적합합니다. |
| `ev_ebitda` | EV/EBITDA | `float64` | `float64` | 밸류에이션 지표이므로 `float64` 타입이 적합합니다. |

In [14]:
dtype_map = {
  'category': ['ticker']
}

for dtype, cols in dtype_map.items():
  # DataFrame에 실제 존재하는 컬럼만 필터링
  valid_cols = [col for col in cols if col in final_df.columns]
  if not valid_cols:
    continue

  try:
    if dtype == 'Int64':
      for col in valid_cols:
        numeric_col = pd.to_numeric(final_df[col], errors='coerce')
        if not numeric_col.isnull().all():
          final_df[col] = numeric_col.astype('Int64')
    
    elif dtype == 'float64':
      final_df[valid_cols] = final_df[valid_cols].apply(pd.to_numeric, errors='coerce')

    elif dtype == 'datetime64':
      for col in valid_cols:
        final_df[col] = pd.to_datetime(final_df[col], errors='coerce')
    
    else: # 'category' 등 .astype()으로 처리 가능한 나머지 타입
      final_df[valid_cols] = final_df[valid_cols].astype(dtype)

  except (ValueError, TypeError) as e:
    print(f"⚠️ 경고: 컬럼 {valid_cols}을(를) '{dtype}'으로 변환 중 오류. 건너뜁니다. (에러: {e})")


In [15]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   ticker                155 non-null    category
 1   period                155 non-null    category
 2   opinion               155 non-null    category
 3   revenue               155 non-null    float64 
 4   revenue_yoy           155 non-null    float64 
 5   operating_profit      155 non-null    float64 
 6   operating_profit_yoy  155 non-null    float64 
 7   net_income            155 non-null    float64 
 8   net_income_yoy        155 non-null    float64 
 9   eps                   154 non-null    float64 
 10  bps                   155 non-null    float64 
 11  per                   155 non-null    float64 
 12  pbr                   140 non-null    float64 
 13  psr                   155 non-null    float64 
 14  roe                   155 non-null    float64 
 15  ev_ebi